# Airline On-Time Statistics and Delay Causes - Data Wrangling

I'm trying to describes the wrangling of the dataset required for the visualization using Tableau within Udacity's Data Analyst Project : "Create a Tableau Story".

As a dataset, we have chosen the Flight Delay Dataset, which can be found [here](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?pn=1) (note: the zip file is downloaded manually since a period of time has to be selected.). The source is the Bureau of Transportation Statistics (BTS) Period limit from June 2003 to Septemper 2018.

In further analysis and visualization, we want to take a look at the data of the recent 8 years to explore temporal trends. Thus, the period of the recent available data has been choosen, in our case from Aug, 2010 to Aug, 2018. Additional data from this site are also required to get more geographical data.

## Gathering

We gather Flights data from the [website](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?pn=1). Therefor, we download the data manually. The archive is then unzipped and converted into a Pandas dataframe programmatically, so we can assess the data in the next step of the data wrangling process.


In [36]:
# Import required modules to the python notebook
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import zipfile
import datetime

%matplotlib inline

In [37]:
# Unzip the PISA archive
zipped_data = zipfile.ZipFile('82404974_92018_162_airline_delay_causes.zip', 'r')
zipped_data.extractall()
zipped_data.close()

In [38]:
# Read CSV file
df_flight = pd.read_csv('82404974_92018_162_airline_delay_causes.zip')
df_flight.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2003,6,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",307.0,56.0,14.68,10.79,...,9.96,1.0,1.0,2530.0,510.0,621.0,676.0,25.0,698.0,NaN
1,2003,6,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",90.0,27.0,7.09,2.00,...,7.16,0.0,0.0,1390.0,271.0,83.0,581.0,0.0,455.0,NaN
2,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,...,17.53,5.0,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0,NaN
3,2003,6,AA,American Airlines Inc.,AUS,"Austin, TX: Austin - Bergstrom International",842.0,174.0,60.24,20.54,...,40.75,9.0,1.0,8344.0,3040.0,1032.0,1835.0,115.0,2322.0,NaN
4,2003,6,AA,American Airlines Inc.,BDL,"Hartford, CT: Bradley International",383.0,55.0,14.90,8.91,...,16.61,0.0,0.0,3137.0,815.0,574.0,555.0,0.0,1193.0,NaN


## Assessing

We look at the dataset in detail to identify structural or content-related issues that affect the data tidiness respectively the data quality.

In [39]:
# View summary of dataset using info()-function
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247718 entries, 0 to 247717
Data columns (total 22 columns):
year                   247718 non-null int64
 month                 247718 non-null int64
carrier                247718 non-null object
carrier_name           247718 non-null object
airport                247718 non-null object
airport_name           247718 non-null object
arr_flights            247368 non-null float64
arr_del15              247317 non-null float64
carrier_ct             247368 non-null float64
 weather_ct            247368 non-null float64
nas_ct                 247368 non-null float64
security_ct            247368 non-null float64
late_aircraft_ct       247368 non-null float64
arr_cancelled          247368 non-null float64
arr_diverted           247368 non-null float64
 arr_delay             247368 non-null float64
 carrier_delay         247368 non-null float64
weather_delay          247368 non-null float64
nas_delay              247368 non-null float64
secu

In [40]:
# Check for NaN values in the "arr_del15" column (variable describes the amount of delayed flights)
df_flight[df_flight["arr_del15"].isnull()]

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
499,2003,6,EV,Atlantic Southeast Airlines,ORF,"Norfolk, VA: Norfolk International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,2003,6,EV,Atlantic Southeast Airlines,SWF,"Newburgh/Poughkeepsie, NY: Stewart International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,2003,6,HP,America West Airlines Inc.,LGB,"Long Beach, CA: Long Beach Airport",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4107,2003,9,DL,Delta Air Lines Inc.,JAC,"Jackson, WY: Jackson Hole",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4761,2003,9,RU,ExpressJet Airlines Inc.,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4767,2003,9,TZ,ATA Airlines d/b/a ATA,ABQ,"Albuquerque, NM: Albuquerque International Sun...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10568,2004,2,EV,Atlantic Southeast Airlines,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10577,2004,2,EV,Atlantic Southeast Airlines,BNA,"Nashville, TN: Nashville International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11333,2004,2,TZ,ATA Airlines d/b/a ATA,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11954,2004,3,EV,Atlantic Southeast Airlines,BNA,"Nashville, TN: Nashville International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Quality issues

- Missing values in some columns

- Date is not given as datetime object

- some column names are misleading

### Data Tidiness issues

- City, State and Airport are in the same column 

- "Unnamed:21" does not contain any data

- Year and Month in different columns

## Cleaning

Define, code and test the cleaning tasks, which have been identified in the assessment section.

### City, State and Airport are in the same column

#### Define
Split column airport_name and save each information in two different columns.

#### Code

In [41]:
# Split string in airport_name
airport_cols = df_flight["airport_name"].str.split(": ",expand=True)
# Create new columns in dataset
df_flight["airport_name"] = airport_cols[1]
df_flight["city"] = airport_cols[0]

#### Test

In [42]:
df_flight.sample(10)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21,city
127161,2010,8,B6,JetBlue Airways,BWI,Baltimore/Washington International Thurgood Ma...,151.0,35.0,14.59,0.00,...,0.0,1.0,2780.0,1288.0,0.0,553.0,0.0,939.0,NaN,"Baltimore, MD"
139940,2011,5,EV,Atlantic Southeast Airlines,SAV,Savannah/Hilton Head International,95.0,28.0,9.85,0.00,...,5.0,0.0,1824.0,644.0,0.0,234.0,0.0,946.0,NaN,"Savannah, GA"
793,2003,6,NW,Northwest Airlines Inc.,PWM,Portland International Jetport,60.0,8.0,2.59,0.00,...,0.0,0.0,196.0,67.0,0.0,129.0,0.0,0.0,NaN,"Portland, ME"
158116,2012,7,US,US Airways Inc.,SJC,Norman Y. Mineta San Jose International,154.0,9.0,4.60,0.00,...,0.0,0.0,556.0,310.0,0.0,102.0,0.0,144.0,NaN,"San Jose, CA"
233537,2018,1,9E,Endeavor Air Inc.,FLL,Fort Lauderdale-Hollywood International,29.0,7.0,2.08,0.00,...,0.0,0.0,496.0,358.0,0.0,98.0,0.0,40.0,NaN,"Fort Lauderdale, FL"
233066,2018,1,OO,SkyWest Airlines Inc.,SBA,Santa Barbara Municipal,349.0,67.0,18.71,0.00,...,5.0,3.0,5193.0,1987.0,0.0,490.0,0.0,2716.0,NaN,"Santa Barbara, CA"
54263,2006,8,WN,Southwest Airlines Co.,ABQ,Albuquerque International Sunport,1723.0,320.0,82.80,4.17,...,2.0,1.0,14565.0,3458.0,161.0,1495.0,303.0,9148.0,NaN,"Albuquerque, NM"
147337,2011,11,AA,American Airlines Inc.,MCI,Kansas City International,308.0,60.0,26.38,1.89,...,0.0,0.0,3075.0,1324.0,104.0,449.0,0.0,1198.0,NaN,"Kansas City, MO"
154626,2012,4,WN,Southwest Airlines Co.,PIT,Pittsburgh International,533.0,77.0,37.23,1.41,...,0.0,0.0,3217.0,1676.0,82.0,166.0,0.0,1293.0,NaN,"Pittsburgh, PA"
17546,2004,7,EV,Atlantic Southeast Airlines,OKC,Will Rogers World,93.0,20.0,8.11,2.54,...,1.0,0.0,989.0,358.0,165.0,217.0,0.0,249.0,NaN,"Oklahoma City, OK"


### "Unnamed:21" does not contain any data

#### Define
The column "Unnamed: 21" has to be dropped.

#### Code

In [43]:
# Drop column "Unnamed: 21"
df_flight.drop(["Unnamed: 21"], axis=1, inplace=True)

#### Test

In [44]:
# View dataset summary
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247718 entries, 0 to 247717
Data columns (total 22 columns):
year                   247718 non-null int64
 month                 247718 non-null int64
carrier                247718 non-null object
carrier_name           247718 non-null object
airport                247718 non-null object
airport_name           247718 non-null object
arr_flights            247368 non-null float64
arr_del15              247317 non-null float64
carrier_ct             247368 non-null float64
 weather_ct            247368 non-null float64
nas_ct                 247368 non-null float64
security_ct            247368 non-null float64
late_aircraft_ct       247368 non-null float64
arr_cancelled          247368 non-null float64
arr_diverted           247368 non-null float64
 arr_delay             247368 non-null float64
 carrier_delay         247368 non-null float64
weather_delay          247368 non-null float64
nas_delay              247368 non-null float64
secu

### Year and Month in different columns

#### Define
Year and Month belong to one variable (date) but are separated in two columns, which is in indicator for 'messy data'.
Both variables have to be concatenated as a string in one column. Since no week or day information are given in the dataset, we assume the first of each month to create a complete date.
Columns Year and Month can be dropped.

#### Code

In [45]:
# put Year and Month together in one column as a string
df_flight["date"] = df_flight["year"].map(str) + "-" + df_flight[" month"].map("{:02}".format) + "-01"

# Drop Year and Month
df_flight.drop(["year"," month"], axis=1, inplace=True)

#### Test

In [46]:
# View Sample of rows
df_flight.sample(10)

,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,city,date
73197,UA,United Air Lines Inc.,PVD,Theodore Francis Green State,155.0,65.0,17.26,2.78,11.18,0.0,...,3.0,0.0,4532.0,1271.0,177.0,452.0,0.0,2632.0,"Providence, RI",2007-08-01
214769,OO,SkyWest Airlines Inc.,CAE,Columbia Metropolitan,3.0,1.0,1.00,0.00,0.00,0.0,...,0.0,0.0,135.0,135.0,0.0,0.0,0.0,0.0,"Columbia, SC",2016-07-01
98144,FL,AirTran Airways Corporation,ATL,Hartsfield-Jackson Atlanta International,6537.0,2135.0,198.63,17.97,891.38,0.0,...,97.0,6.0,106590.0,9691.0,1270.0,37034.0,0.0,58595.0,"Atlanta, GA",2008-12-01
36086,MQ,American Eagle Airlines Inc.,OKC,Will Rogers World,271.0,55.0,21.72,5.33,11.86,0.0,...,2.0,0.0,2470.0,990.0,263.0,312.0,0.0,905.0,"Oklahoma City, OK",2005-08-01
204138,MQ,Envoy Air,JFK,John F. Kennedy International,165.0,39.0,14.67,0.07,13.26,1.0,...,7.0,0.0,2518.0,1188.0,3.0,692.0,56.0,579.0,"New York, NY",2015-09-01
1077,UA,United Air Lines Inc.,GRR,Gerald R. Ford International,109.0,30.0,5.31,0.11,14.20,0.0,...,0.0,0.0,1290.0,274.0,10.0,421.0,0.0,585.0,"Grand Rapids, MI",2003-06-01
191669,OO,SkyWest Airlines Inc.,ELP,El Paso International,240.0,25.0,10.73,0.70,7.27,0.0,...,1.0,0.0,1132.0,413.0,45.0,179.0,0.0,495.0,"El Paso, TX",2014-10-01
58999,YV,Mesa Airlines Inc.,FLG,Flagstaff Pulliam,146.0,18.0,18.00,0.00,0.00,0.0,...,0.0,0.0,1082.0,1082.0,0.0,0.0,0.0,0.0,"Flagstaff, AZ",2006-11-01
111997,UA,United Air Lines Inc.,PDX,Portland International,368.0,33.0,7.60,0.10,4.14,0.0,...,1.0,0.0,2318.0,752.0,9.0,172.0,0.0,1385.0,"Portland, OR",2009-09-01
156433,F9,Frontier Airlines Inc.,HOU,William P Hobby,2.0,2.0,1.01,0.00,0.06,0.0,...,0.0,0.0,274.0,71.0,0.0,12.0,0.0,191.0,"Houston, TX",2012-06-01


### Date is not given as datetime object

#### Define
The created date column is given as string and has to be converted into datetime object.

#### Code

In [47]:
# Format date to datetime object
df_flight["date"] = pd.to_datetime(df_flight["date"], format='%Y-%m-%d')

#### Test

In [48]:
# Check info()-function for datatype of column date
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 247718 entries, 0 to 247717
Data columns (total 21 columns):
carrier                247718 non-null object
carrier_name           247718 non-null object
airport                247718 non-null object
airport_name           247718 non-null object
arr_flights            247368 non-null float64
arr_del15              247317 non-null float64
carrier_ct             247368 non-null float64
 weather_ct            247368 non-null float64
nas_ct                 247368 non-null float64
security_ct            247368 non-null float64
late_aircraft_ct       247368 non-null float64
arr_cancelled          247368 non-null float64
arr_diverted           247368 non-null float64
 arr_delay             247368 non-null float64
 carrier_delay         247368 non-null float64
weather_delay          247368 non-null float64
nas_delay              247368 non-null float64
security_delay         247368 non-null float64
late_aircraft_delay    247368 non-null float64


### Missing values in some columns

#### Define
Some delay information are missing in the dataset. Corresponding rows are useless for the analysis and visualization of flight delays or cancellations. Thus, these observations can be dropped.

#### Code

In [49]:
# Drop rows with NaN in the "arr_flight" column
df_flight.dropna(inplace=True)

#### Test

In [50]:
# Check the info()-function - all columns have to be exactly the same amount of non-null values
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247317 entries, 0 to 247717
Data columns (total 21 columns):
carrier                247317 non-null object
carrier_name           247317 non-null object
airport                247317 non-null object
airport_name           247317 non-null object
arr_flights            247317 non-null float64
arr_del15              247317 non-null float64
carrier_ct             247317 non-null float64
 weather_ct            247317 non-null float64
nas_ct                 247317 non-null float64
security_ct            247317 non-null float64
late_aircraft_ct       247317 non-null float64
arr_cancelled          247317 non-null float64
arr_diverted           247317 non-null float64
 arr_delay             247317 non-null float64
 carrier_delay         247317 non-null float64
weather_delay          247317 non-null float64
nas_delay              247317 non-null float64
security_delay         247317 non-null float64
late_aircraft_delay    247317 non-null float64


### Some column names are misleading

#### Define
Rename column names using a dictionary.

#### Code

In [51]:
# Define a dictionary and rename columns
new_cols = {"carrier":"Carrier",
            "carrier_name": "Carrier_Name",
            "airport":"Airport",
            "airport_name":"Airport_Name",
            "arr_flights":"Flights_count",
            "arr_del15":"Delays_count",
            "carrier_ct":"Carrier_Delays_count",
            " weather_ct":"Weather_Delays_count",
            "nas_ct":"NAS_Delays_count",
            "security_ct":"Security_Delays_count",
            "late_aircraft_ct":"Late_Aircraft_Delays_count",
            "arr_cancelled":"Cancellations_count",
            "arr_diverted":"Diversions_count",
            " arr_delay":"Delay_minutes",
            " carrier_delay":"Carrier_Delay_minutes",
            "weather_delay":"Weather_Delay_minutes",
            "nas_delay":"NAS_Delay_minutes",
            "security_delay":"Security_Delay_minutes",
            "late_aircraft_delay":"Late_Aircraft_Delay_minutes",
            "city":"City",
            "date":"Date"}
df_flight.rename(index=str, columns=new_cols,inplace=True)

#### Test

In [52]:
df_flight.info()

<class 'pandas.core.frame.DataFrame'>
Index: 247317 entries, 0 to 247717
Data columns (total 21 columns):
Carrier                        247317 non-null object
Carrier_Name                   247317 non-null object
Airport                        247317 non-null object
Airport_Name                   247317 non-null object
Flights_count                  247317 non-null float64
Delays_count                   247317 non-null float64
Carrier_Delays_count           247317 non-null float64
Weather_Delays_count           247317 non-null float64
NAS_Delays_count               247317 non-null float64
Security_Delays_count          247317 non-null float64
Late_Aircraft_Delays_count     247317 non-null float64
Cancellations_count            247317 non-null float64
Diversions_count               247317 non-null float64
Delay_minutes                  247317 non-null float64
Carrier_Delay_minutes          247317 non-null float64
Weather_Delay_minutes          247317 non-null float64
NAS_Delay_minutes 

## Storage

We store the dataset as a new Excel-file.

In [53]:
# Save to Excel-file
df_flight.to_excel('Airline On-Time Statistics and Delay Causes.xlsx',index=False)